In [1]:
# !pip install transformers accelerate
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
# !pip install datasets
from datasets import load_metric
# %pip install evaluate
from evaluate import evaluator
from sklearn.model_selection import train_test_split
# from ipywidgets import FloatProgress
import csv
from optuna import Trial
from typing import Dict, Union, Any
import os
import sys
# notebook_login()

csv.field_size_limit(500 * 1024 * 1024)
CUDA_LAUNCH_BLOCKING=1
# get pwd
notebook_path = os.path.abspath('')

# Find the part of the path that contains 'commitFit'
commit_fit_path = None
for part in notebook_path.split(os.sep):
    print(part)
    if 'CommitFit' in part:
        commit_fit_path = notebook_path.split(part)[0] + part
        break

if commit_fit_path is None:
    raise ValueError("Path containing 'commitFit' not found in notebook path.")

# Add commitFit directory to Python path, so we can import moudule from commitfit folder directly
if commit_fit_path not in sys.path:
    sys.path.append(commit_fit_path)

from commitfit import CommitFitModel, CommitFitTrainer 


CommitFit


In [2]:
# from google.colab import drive
# drive.mount('/content/drive') ['Corrective','Adaptive','Perfective']
label2id = {'a':'Adaptive','p':'Perfective','c':'Corrective'}

In [3]:
# df = pd.read_csv('D:\IET software-CC-CL\Commit Classification\experiment\Commit_dataset.csv', encoding="cp1252")

# label2id = {'a':'Adaptive','p':'Perfective','c':'Corrective'}
# df = df.replace({"3_labels": label2id})
# df = pd.read_csv(r'dataset.csv',engine="python")
# df['text'] = "['DIFF]" + df['diff']
# # df = df.replace({"2_labels": label2id})
# print(df)

train = pd.read_csv('train.csv',index_col=0)
train = train.rename(columns={'3_labels':'label','comment':'text'})
train.fillna(0, inplace=True)
test = pd.read_csv('test.csv',index_col=0)
test = test.rename(columns={'3_labels':'label','comment':'text'})
test.fillna(0, inplace=True)
print(len(test))



1758


In [4]:
# df

In [5]:
# 先来10条测试代码
# train = df.sample(n=5, random_state=1)
# train = df.groupby('3_labels').apply(lambda s: s.sample(5)).reset_index(drop=True)
# df
# train

In [6]:
from commitfit import get_templated_dataset,sample_dataset
from datasets import Dataset, load_metric

In [7]:
# train = df.rename(columns={'3_labels':'label','comment':'text'})

In [8]:
Dataset_train = Dataset.from_pandas(train)
test_dataset = Dataset.from_pandas(test)


In [9]:
train_dataset = get_templated_dataset(Dataset_train, candidate_labels=['Corrective','Adaptive','Perfective'], sample_size=8)

In [10]:
# list(train['message'].astype(str).values)
train_dataset

Dataset({
    features: ['commit_id', 'project', 'text', 'label', 'ADDING_ATTRIBUTE_MODIFIABILITY', 'ADDING_METHOD_OVERRIDABILITY', 'ADDITIONAL_CLASS', 'ADDITIONAL_FUNCTIONALITY', 'ADDITIONAL_OBJECT_STATE', 'ALTERNATIVE_PART_DELETE', 'ALTERNATIVE_PART_INSERT', 'ATTRIBUTE_RENAMING', 'ATTRIBUTE_TYPE_CHANGE', 'CLASS_RENAMING', 'COMMENT_DELETE', 'COMMENT_INSERT', 'COMMENT_MOVE', 'COMMENT_UPDATE', 'CONDITION_EXPRESSION_CHANGE', 'DECREASING_ACCESSIBILITY_CHANGE', 'DOC_DELETE', 'DOC_INSERT', 'DOC_UPDATE', 'INCREASING_ACCESSIBILITY_CHANGE', 'METHOD_RENAMING', 'PARAMETER_DELETE', 'PARAMETER_INSERT', 'PARAMETER_ORDERING_CHANGE', 'PARAMETER_RENAMING', 'PARAMETER_TYPE_CHANGE', 'PARENT_CLASS_CHANGE', 'PARENT_CLASS_DELETE', 'PARENT_CLASS_INSERT', 'PARENT_INTERFACE_CHANGE', 'PARENT_INTERFACE_DELETE', 'PARENT_INTERFACE_INSERT', 'REMOVED_CLASS', 'REMOVED_FUNCTIONALITY', 'REMOVED_OBJECT_STATE', 'REMOVING_ATTRIBUTE_MODIFIABILITY', 'REMOVING_CLASS_DERIVABILITY', 'REMOVING_METHOD_OVERRIDABILITY', 'RETURN_T

In [11]:
len(train)

35

In [12]:
# encoded_train = tokenizer(train_dataset['text'].astype(str).to_list(), return_tensors='pt',truncation=True, padding=True)
# print(encoded_train["input_ids"].shape)
# encoded_test = tokenizer(test['comment'].astype(str).to_list(), return_tensors='pt',truncation=True, padding=True)
# print(encoded_test["input_ids"].shape)
# # encoded_val = tokenizer(val['comment'].astype(str).to_list(), return_tensors='pt',truncation=True, padding='max_length')

In [13]:
# encoded_train

In [14]:
train['label'].value_counts()

label
Corrective    12
Perfective    12
Adaptive      11
Name: count, dtype: int64

In [15]:
test['label'].value_counts()

label
Corrective    591
Perfective    588
Adaptive      579
Name: count, dtype: int64

In [16]:
# huggingface-cli login
# train_dataset = CommitDataset(encoded_train, list(train['3_labels']))
# test_dataset = CommitDataset(encoded_test, list(test['3_labels']))
# val_dataset = CommitDataset(encoded_val, list(val['label']))

In [17]:
len(train_dataset)

59

In [18]:
from sklearn import  metrics

def compute_metrics(y_pred, y_test):
    # print(y_pred,y_test)
    # classification_report = metrics.classification_report(y_test,y_pred)
    accuracy_score = metrics.accuracy_score(y_test,y_pred)
    precision_score = metrics.precision_score(y_test,y_pred, average='weighted')
    recall_score = metrics.recall_score(y_test,y_pred,average='weighted')
    f1_score = metrics.f1_score(y_test,y_pred,average='weighted')

    # return {"classification_report": classification_report}
    return {"precision": precision_score,"recall": recall_score, "f1":f1_score, "accuracy": accuracy_score}

In [19]:
train_dataset

Dataset({
    features: ['commit_id', 'project', 'text', 'label', 'ADDING_ATTRIBUTE_MODIFIABILITY', 'ADDING_METHOD_OVERRIDABILITY', 'ADDITIONAL_CLASS', 'ADDITIONAL_FUNCTIONALITY', 'ADDITIONAL_OBJECT_STATE', 'ALTERNATIVE_PART_DELETE', 'ALTERNATIVE_PART_INSERT', 'ATTRIBUTE_RENAMING', 'ATTRIBUTE_TYPE_CHANGE', 'CLASS_RENAMING', 'COMMENT_DELETE', 'COMMENT_INSERT', 'COMMENT_MOVE', 'COMMENT_UPDATE', 'CONDITION_EXPRESSION_CHANGE', 'DECREASING_ACCESSIBILITY_CHANGE', 'DOC_DELETE', 'DOC_INSERT', 'DOC_UPDATE', 'INCREASING_ACCESSIBILITY_CHANGE', 'METHOD_RENAMING', 'PARAMETER_DELETE', 'PARAMETER_INSERT', 'PARAMETER_ORDERING_CHANGE', 'PARAMETER_RENAMING', 'PARAMETER_TYPE_CHANGE', 'PARENT_CLASS_CHANGE', 'PARENT_CLASS_DELETE', 'PARENT_CLASS_INSERT', 'PARENT_INTERFACE_CHANGE', 'PARENT_INTERFACE_DELETE', 'PARENT_INTERFACE_INSERT', 'REMOVED_CLASS', 'REMOVED_FUNCTIONALITY', 'REMOVED_OBJECT_STATE', 'REMOVING_ATTRIBUTE_MODIFIABILITY', 'REMOVING_CLASS_DERIVABILITY', 'REMOVING_METHOD_OVERRIDABILITY', 'RETURN_T

In [20]:
model_id = "../../sentence-transformers/all-roberta-large-v1"

In [21]:
def hp_space(trial: Trial) -> Dict[str, Union[float, int, str]]:
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-3, log=True),
        # "num_epochs": trial.suggest_int("num_epochs", 1, 3),
        # "batch_size": trial.suggest_categorical("batch_size", [8, 12, 16]),
        # "seed": trial.suggest_int("seed", 1, 40),
        # "num_iterations": trial.suggest_int("num_iterations", 10, 20),
        # "solver": trial.suggest_categorical("solver", ["newton-cg", "lbfgs", "liblinear"]),
    }
def model_init(params: Dict[str, Any]) -> CommitFitModel:
    params = params or {}
    # learning_rate = params.get("learning_rate")
    # num_iterations = params.get("num_iterations", 20)
    # solver = params.get("solver", "liblinear")
    # params = {
    #     "head_params": {
    #         # "max_iter": num_iterations,
    #         # "solver": solver,
    #     }
    # }
    return CommitFitModel.from_pretrained(model_id, **params)
def my_compute_objective(metrics):
    print('+++++++++++',metrics)
    return  metrics['accuracy']

In [22]:

trainer = CommitFitTrainer(
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    model_init=model_init,
    metric = compute_metrics,
    num_iterations=20,
    num_epochs=1
)
best_run = trainer.hyperparameter_search(direction="maximize", hp_space=hp_space, compute_objective=my_compute_objective, n_trials=10)

model_head.pkl not found in /CommitFit/sentence-transformers/all-roberta-large-v1, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
[I 2024-05-26 17:47:08,770] A new study created in memory with name: no-name-53b94d7c-929d-4264-a9ad-16f9cb5cdd79
Trial: {'learning_rate': 1.1336962412748385e-05}
model_head.pkl not found in /CommitFit/sentence-transformers/all-roberta-large-v1, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


Generating Training Pairs:   0%|          | 0/20 [00:00<?, ?it/s]

***** Running training *****
  Num examples = 2360
  Num epochs = 1
  Total optimization steps = 148
  Total train batch size = 16


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/148 [00:00<?, ?it/s]

...

***** Running training *****
  Num examples = 2360
  Num epochs = 1
  Total optimization steps = 148
  Total train batch size = 16


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/148 [00:00<?, ?it/s]

***** Running evaluation *****
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[I 2024-05-26 17:56:15,212] Trial 9 finished with value: 0.33447098976109213 and parameters: {'learning_rate': 0.0003344683859640846}. Best is trial 4 with value: 0.626848691695108.


+++++++++++ {'precision': 0.1118708429917646, 'recall': 0.33447098976109213, 'f1': 0.16766320714366767, 'accuracy': 0.33447098976109213}


In [23]:
best_run

BestRun(run_id='4', objective=0.626848691695108, hyperparameters={'learning_rate': 1.69354119307237e-05}, backend=<optuna.study.study.Study object at 0x7f489608e850>)

In [24]:
best_run.hyperparameters

{'learning_rate': 1.69354119307237e-05}

In [25]:
trainer.apply_hyperparameters(best_run.hyperparameters, final_model=True)
trainer.train()

model_head.pkl not found in /CommitFit/sentence-transformers/all-roberta-large-v1, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


Generating Training Pairs:   0%|          | 0/20 [00:00<?, ?it/s]

***** Running training *****
  Num examples = 2360
  Num epochs = 1
  Total optimization steps = 148
  Total train batch size = 16


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/148 [00:00<?, ?it/s]

In [26]:
fewshot_metrics = trainer.evaluate()
fewshot_metrics

***** Running evaluation *****


{'precision': 0.5942352986297639,
 'recall': 0.5915813424345847,
 'f1': 0.586700923997796,
 'accuracy': 0.5915813424345847}